## Package Imports

In [24]:
import numpy as np
import itertools
import time
import matplotlib.pyplot as plt

## The Search Algorithm

In [25]:
def calculate_euclidean_distance(x1, y1, x2, y2):
    """
    Takes 2 pairs of coordinates and calculates the Euclidean distance between them.

    Args:
        - x1 (int): x-coordinate of city 1.
        - y1 (int): y-coordinate of city 1.
        - x2 (int): x-coordinate of city 2.
        - y2 (int): y-coordinate of city 2.

    Returns:
        - distance (float): Euclidean distance between city 1 and city 2.
    """
    x_dist = abs(x2 - x1)
    y_dist = abs(y2 - y1)

    distance = np.sqrt(x_dist ** 2 + y_dist ** 2)

    return distance



def load_problem_instance(filename):
    """
    Takes a .vrp file generated by Uchoa et al. (2013) and converts the contents of the 
    problem instance into a dictionary with useful information stored accordingly.

    Args:
        - filename (str): string containing the filename of the problem instance.

    Returns:
        - problem_instance (dict):
            - name (str): unique problem instance name.
            - type (str): problem instance category.
            - dimension (int): the number of nodes (cities) in the problem instance.
                Includes the depot.
            - k_min (int): the minimum number of routes required by the problem instance.
            - edge_weight_type (str): the method of generating city coordinates.
            - capacity (int): the maximum capacity that each route may transport.
            - node_coordinates (np.array([list])): a numpy array of length: dimension, with
                each object a list of shape: [city_number, x_coordinate, y_coordinate].
            - demand_vector (np.array([list])): a numpy array of length: dimension, with
                each object a list of shape: [city_number, demand].
            - distance_matrix (np matrix): a numpy matrix of shape: dimension x dimension, 
                describing the pairwise distance between each city.
    """
    problem_instance = {
        "name": "",
        "type": "",
        "dimension": "",
        "k_min": "",
        "edge_weight_type": "",
        "capacity": "",
        "node_coordinates": "",
        "demand_vector": "",
        "distance_matrix": ""
    }

    with open(filename, "r") as file:
        flag_reading_coordinates = False
        flag_reading_demand = False

        node_coordinates = []
        demand_vector = []

        lines = file.readlines()
        for line in lines:
            if "DEPOT_SECTION" in line:
                break

            # Set flag for reading node coordinates if beginning this section
            elif "NODE_COORD_SECTION" in line:
                flag_reading_coordinates = True
            
            # Set flag for reading demand vector if beginning this section
            elif "DEMAND_SECTION" in line:
                flag_reading_coordinates = False
                flag_reading_demand = True
            
            # If in node coordinates section of file, extract node coordinates
            elif flag_reading_coordinates == True:
                node = []
                node.append(int(line.split("\t")[0])) # Node ID
                node.append(int(line.split("\t")[1])) # Node x-coordinate
                node.append(int(line.split("\t")[2])) # Node y-coordinate
                node_coordinates.append(node)
            
            # If in demand vector section of file, extract demand vector
            elif flag_reading_demand == True:
                demand = []
                demand.append(int(line.split("\t")[0])) # Demand ID
                demand.append(int(line.split("\t")[1])) # Demand value
                demand_vector.append(demand)

            # Strip the problem instance name from line 1
            elif "NAME" in line:
                problem_instance["name"] = line.replace("NAME : \t", "").strip()
                
                # Extract k_min from the problem instance name
                problem_instance["k_min"] = int(problem_instance["name"].split("k")[-1])

            # Strip the problem instance edge weight type from line 5
            elif "EDGE_WEIGHT_TYPE" in line:
                problem_instance["edge_weight_type"] = line.replace("EDGE_WEIGHT_TYPE : \t", "").strip()

            # Strip the problem instance type from line 3
            elif "TYPE" in line:
                problem_instance["type"] = line.replace("TYPE : \t", "").strip()

            # Strip the problem instance dimension from line 4
            elif "DIMENSION" in line:
                problem_instance["dimension"] = int(line.replace("DIMENSION : \t", "").strip())

            # Strip the problem instance capacity from line 6
            elif "CAPACITY" in line:
                problem_instance["capacity"] = int(line.replace("CAPACITY : \t", "").strip())
    
        # Insert node coordinates and distance vector into problem instance dictionary
        problem_instance["node_coordinates"] = np.array(node_coordinates)
        problem_instance["demand_vector"] = np.array(demand_vector)

    # Create empty distance matrix from node coordinates array
    distance_matrix = np.zeros((problem_instance["dimension"], problem_instance["dimension"]))

    # Calculate distances
    for i in range(problem_instance["dimension"]):
        i_x_coordinate = problem_instance["node_coordinates"][i][1]
        i_y_coordinate = problem_instance["node_coordinates"][i][2]

        for j in range(problem_instance["dimension"]):
            j_x_coordinate = problem_instance["node_coordinates"][j][1]
            j_y_coordinate = problem_instance["node_coordinates"][j][2]

            if i > j:
                continue
                
            elif i == j:
                distance_matrix[i, j] = 0
                
            elif i < j:
                # Calculate distance only once for each node pair
                calculated_distance = calculate_euclidean_distance(
                    i_x_coordinate, i_y_coordinate, j_x_coordinate, j_y_coordinate
                )

                # Mirror the calculated distance for both halves of matrix
                distance_matrix[i, j] = calculated_distance
                distance_matrix[j, i] = calculated_distance

    problem_instance["distance_matrix"] = distance_matrix

    return problem_instance


In [26]:
def decode_solution(problem_instance, chromosome):
    """
    Used to convert a chromosome into a viable set of routes according to the defined
    problem instance. 
    Works by assigning each consecutive city to the existing route if it does not cause
    the capacity of that route to be exceeded, or to a new route if it does cause the
    capacity of that route to be exceeded.

    Args:
        - problem_instance (dict): a dictionary containing key metadata for the defined
            problem instance.
        - chromosome (list): a list containing the permutation order of cities for a
            solution. Has length: (problem_instance["dimension"] - 1).

    Returns:
        - routes (list[lists]): a list of lists containing the decoded set of routes that
            is unique to a chromosome.
        - K (int): the number of routes, equal to len(routes).
    """
    CAPACITY = problem_instance["capacity"]
    DEMAND_VECTOR = problem_instance["demand_vector"]

    routes = []
    route_weights = []

    for i in range(len(chromosome)):
        # Get city no. and demand for ith city in solution chromosome
        next_city = chromosome[i]
        next_city_weight = DEMAND_VECTOR[next_city - 1][1]

        # Initiate current route vector and weight if at start of iteration
        if i == 0:
            current_route = []
            current_route.append(1) # Route must start from depot
            current_route_weight = 0
        
        # If next city does not cause capacity to be exceeded, update route and route weight
        if (current_route_weight + next_city_weight) <= CAPACITY:
            current_route.append(next_city)
            current_route_weight += next_city_weight

            # Check if final city on chromosome. If so, close the last route
            if i == len(chromosome) - 1:
                # Update list of routes and route weights
                current_route.append(1) # Route must end at depot
                routes.append(current_route)
                route_weights.append(current_route_weight)

        # If next city DOES cause capacity to be exceeded:
        else:
            # Update list of routes and route weights
            current_route.append(1) # Route must end at depot
            routes.append(current_route)
            route_weights.append(current_route_weight)

            # Reset route and weight for the start of the next route
            current_route = []
            current_route.append(1)
            current_route_weight = 0

            # Add the first city of the new route and update weight
            current_route.append(next_city)
            current_route_weight += next_city_weight

            # Check if final city on chromosome. If so, close the last route
            if i == len(chromosome) - 1:
                # Update list of routes and route weights
                current_route.append(1) # Route must end at depot
                routes.append(current_route)
                route_weights.append(current_route_weight)
    
    K = len(routes)

    return routes, K


In [27]:
def evaluate_solution(problem_instance, routes):
    """
    Used to take the unique list of routes generated by decoding a chromosome and calculate
    the total distance travelled - the cost of that solution.

    Args:
        - problem_instance (dict): a dictionary containing key metadata for the defined
            problem instance.
        - routes (list[lists]): a list of lists containing the decoded set of routes that
            is unique to a chromosome.

    Returns:
        - total_cost (float): the total distance travelled by the routes unique to a solution
            chromosome.
    """
    DISTANCE_MATRIX = problem_instance["distance_matrix"]
    
    total_cost = 0

    for route in routes:
        for i in range(len(route) - 1):
            # sums the cost of route i, adding it to the total solution cost
            cost = DISTANCE_MATRIX[route[i] - 1, route[i + 1] - 1]
            total_cost += cost

    return total_cost


In [28]:
def generate_random_population(problem_instance, pop_size):
    """
    Used to generate a list of unique, randomly generated solutions to the problem instance.
    Calls decode_solution() and evaluate_solution() in order to produce the cost of the 
    solution as and when the solution is generated.

    Args:
        - problem_instance (dict): a dictionary containing key metadata for the defined
            problem instance.
        - pop_size (int): the number of solutions to be generated.

    Returns:
        - population (list[solutions]):
            - solution (dict):
                - chromosome (list): a list containing the permutation order of cities for 
                    a solution. Has length: (problem_instance["dimension"] - 1).
                - routes (list[lists]): a list of lists containing the decoded set of routes 
                    that is unique to a chromosome.
                - k (int): the number of routes, equal to len(routes).
                - cost (float): the total distance travelled by the routes unique to a 
                    solution chromosome.
    """
    DIMENSION = problem_instance["dimension"]
    K_MIN = problem_instance["k_min"]

    # Generates an unshuffled permutation to be continuously shuffled in solution generation
    unshuffled_chromosome = np.arange(2, DIMENSION + 1)

    population = []

    for i in range(pop_size):
        # Creates a chromosome via random shuffling - converted from 
        shuffled_chromosome = np.random.permutation(unshuffled_chromosome).tolist()

        # Initialise an empty solution dictionary with the random chromosome
        solution = {
            "chromosome": shuffled_chromosome,
            "routes": "",
            "k": "",
            "cost": ""
        }

        # Decodes the chromosome into a viable set of routes
        routes, k = decode_solution(problem_instance, solution["chromosome"])
        solution["routes"] = routes
        solution["k"] = k

        # Evaluates the cost (total distance travelled) of the decoded routes
        cost = evaluate_solution(problem_instance, routes)
        solution["cost"] = cost

        population.append(solution)

    return population


In [29]:
def generate_NN_population(problem_instance):
    """
    Used to generate a list of unique solutions to the problem instance generated via a 
    greedy, nearest neighbour algorithm.
    Calls decode_solution() and evaluate_solution() in order to produce the cost of the 
    solution as and when the solution is generated.

    Args:
        - problem_instance (dict): a dictionary containing key metadata for the defined
            problem instance.
        - pop_size (int): the number of solutions to be generated.

    Returns:
        - population (list[solutions]):
            - solution (dict):
                - chromosome (list): a list containing the permutation order of cities for 
                    a solution. Has length: (problem_instance["dimension"] - 1).
                - routes (list[lists]): a list of lists containing the decoded set of routes 
                    that is unique to a chromosome.
                - k (int): the number of routes, equal to len(routes).
                - cost (float): the total distance travelled by the routes unique to a 
                    solution chromosome.
    """
    DIMENSION = problem_instance["dimension"]
    CAPACITY = problem_instance["capacity"]
    DEMAND_VECTOR = problem_instance["demand_vector"]  
    DISTANCE_MATRIX = problem_instance["distance_matrix"]

    population = []

    for start_city in range(2, DIMENSION + 1):
        # Initialise empty vector for storing the growing solution
        NN_chromosome = []
        # Initialise an empty van
        current_van_weight = 0

        # Update our solution with the start_city and start_city demand
        NN_chromosome.append(start_city)
        current_van_weight += DEMAND_VECTOR[start_city - 1][1]

        previous_city = start_city

        # Repeat until the solution is the correct length
        while len(NN_chromosome) < DIMENSION - 1:
            # Collect the distances between the previous city and all other cities
            city_distances = []
            city_distances = DISTANCE_MATRIX[previous_city - 1,].copy()
            city_distances[0] = float("inf") # Ensure it cannot select the depot as the nearest city

            # Ensure it cannot select any visited city as the nearest city
            for visited_city in NN_chromosome:
                city_distances[visited_city - 1] = float("inf")
        
            city_added = False
            # Try up to 5 times to add the next nearest city without breaching van capacity
            for i in range(5):
                # Try the next nearest city
                nearest_city_index = np.argmin(city_distances)

                # If this causes the current route capacity to be exceeded, rule out this city
                if current_van_weight + DEMAND_VECTOR[nearest_city_index][1] >= CAPACITY:
                    city_distances[nearest_city_index] = float("inf")
                # Failsafe to ensure it never considers a disallowed city
                elif city_distances[nearest_city_index] == float("inf"):
                    continue
                else: # If adding the city does not cause the route capacity to be exceeded
                    # Add this city to the solution, and update the route weight
                    nearest_city = nearest_city_index + 1
                    NN_chromosome.append(nearest_city)
                    current_van_weight += DEMAND_VECTOR[nearest_city_index][1]

                    # Ensures the loops update correctly
                    previous_city = nearest_city
                    city_added = True
                    break
        
            # If a city could not be added in 5 tries, start the next route
            if city_added == False:
                # Start next route by resetting van weight to 0
                current_van_weight = 0
        
        # Initialise an empty solution dictionary with the NN chromosome
        solution = {
            "chromosome": NN_chromosome,
            "routes": "",
            "k": "",
            "cost": ""
        }

        # Decodes the chromosome into a viable set of routes
        routes, k = decode_solution(problem_instance, solution["chromosome"])
        solution["routes"] = routes
        solution["k"] = k

        # Evaluates the cost (total distance travelled) of the decoded routes
        cost = evaluate_solution(problem_instance, routes)
        solution["cost"] = cost

        population.append(solution)
    
    return population


In [30]:
def parent_selection(problem_instance, pop_size, tournament_bias, number_of_elites, elites, non_elites):
    """
    Used to select 2 parents from the total population for crossover.
        - Parent 1 is selected at random from the elite subpopulation.
        - Parent 2 is selected via a biased binary tournament between two non-elite solutions.

    Args:
        - problem_instance (dict): a dictionary containing key metadata for the defined
            problem instance.
        - pop_size (int): the number of solutions to be generated.
        - tournament_bias (float): the preference towards selecting the fitter solution as 
            parent_2.
        - number_of_elites (int): the total number of solutions designated as elite. Calculated
            as pop_size * elitism rate.
        - elites (list[solutions]): the subpopulation of solutions designated as elite.
        - non_elites (list[solutions]): the subpopulation of solutions designated as non-elite.

    Returns:
        - population (list[solutions]):
            - solution (dict):
                - chromosome (list): a list containing the permutation order of cities for 
                    a solution. Has length: (problem_instance["dimension"] - 1).
                - routes (list[lists]): a list of lists containing the decoded set of routes 
                    that is unique to a chromosome.
                - k (int): the number of routes, equal to len(routes).
                - cost (float): the total distance travelled by the routes unique to a 
                    solution chromosome.
    """
    # Select parent 1 at random from the elite group
    parent_1 = elites[np.random.randint(0, number_of_elites)]

    # Select parent 2 via probabilistic binary tournament selection on non-elites
    tournament_indices = np.random.choice(np.arange(0, pop_size - number_of_elites), 2, replace = False)
    tournament_option_1 = non_elites[tournament_indices[0]]
    option_1_cost = tournament_option_1["cost"]
    tournament_option_2 = non_elites[tournament_indices[1]]
    option_2_cost = tournament_option_2["cost"]

    # Check a random float against the tournament bias parameter
    if np.random.random() <= tournament_bias:
        # Bias towards selecting the fitter individual as parent 2
        if option_1_cost <= option_2_cost:
            parent_2 = tournament_option_1
        else:
            parent_2 = tournament_option_2
    else:
        # Bias against selecting the fitter individual as parent 2
        if option_1_cost <= option_2_cost:
            parent_2 = tournament_option_2
        else:
            parent_2 = tournament_option_1
    
    return parent_1, parent_2


In [31]:
def perform_crossover(problem_instance, parent1, parent2):
    """
    Used to take 2 parent chromosomes and cross them over to create a single offspring 
    chromosome.
    
    Performs crossover by first selecting 2 cut points at random. Cities from parent 1 are 
    inserted in order between those cut points according to the original positions in parent 
    1. The remainder of the offspring chromosome is generated from parent 2. This is done by 
    selecting cities from parent 2. They are selected from the point of the final city added 
    to the offspring parent, and added them to the offspring if they are yet to appear. This 
    nsures all offspring are valid permutation solutions.

    Args:
        - problem_instance (dict): a dictionary containing key metadata for the defined
            problem instance.
        - parent1 (dict): the solution selected to be parent 1.
        - parent2 (dict): the solution selected to be parent 2.

    Returns:
        - offspring (list): a list containing the permutation order of cities for a
            solution. Has length: (problem_instance["dimension"] - 1).
    """
    CHROMOSOME_LENGTH = problem_instance["dimension"] - 1

    # Have parents be chromosome 1 or 2 with equal bias
    if np.random.random() < 0.5:
        chromosome1 = parent1["chromosome"]
        chromosome2 = parent2["chromosome"]
    else:
        chromosome1 = parent2["chromosome"]
        chromosome2 = parent1["chromosome"]

    # Initialise empty offspring chromosome
    offspring = [None for _ in range(CHROMOSOME_LENGTH)]
    
    # Select two cut points at random
    cut_points = np.sort(np.random.choice(np.arange(0, CHROMOSOME_LENGTH), 2, replace = False))

    # To offspring, add from chromosome1 all values between the cut points in order
    for i in range(cut_points[0], cut_points[1]):
        offspring[i] = chromosome1[i]
        
        if i == cut_points[1] - 1:
            last_added_city = chromosome1[i] # Store the last added city
        
    # Fill the remainder of offspring chronologically from chromosome2
    # Note this is initiated from the last added city's location in chromosome2

    for i in range(cut_points[1], CHROMOSOME_LENGTH):
        # Get next possible city to add from chromosome 2
        last_added_city_index = chromosome2.index(last_added_city)
        next_possible_city_index = last_added_city_index + 1

        # Try adding the next city until the next city to add is unused
        route_extended = False
        while route_extended == False:
            # Wrap around to start of chromosome2 when you reach the end
            if next_possible_city_index == CHROMOSOME_LENGTH:
                next_possible_city_index = 0

            if chromosome2[next_possible_city_index] not in offspring:
                offspring[i] = chromosome2[next_possible_city_index]
                last_added_city = offspring[i]
                route_extended = True # Break while loop
            else:
                # If city is already in offspring, try to the next city
                next_possible_city_index += 1

    for i in range(0, cut_points[0]):
        # Get next possible city to add from chromosome 2
        last_added_city_index = chromosome2.index(last_added_city)
        next_possible_city_index = last_added_city_index + 1

        # Try adding the next city until the next city to add is unused
        route_extended = False
        while route_extended == False:
            # Wrap around to start of chromosome2 when you reach the end
            if next_possible_city_index == CHROMOSOME_LENGTH:
                next_possible_city_index = 0

            if chromosome2[next_possible_city_index] not in offspring:
                offspring[i] = chromosome2[next_possible_city_index]
                last_added_city = offspring[i]
                route_extended = True # Break while loop
            else:
                # If city is already in offspring, try to the next city
                next_possible_city_index += 1

    return offspring


In [32]:
def perform_inversion_mutation(problem_instance, parent, start_index):
    """
    Used to take a single parent chromosome and perform an inversion mutation, returning
    the offspring solution as a result.

    The inversion is initiated at start_index. A random inversion length is generated, and
    and end_index calculated. The order of cities from start_index to end_index is reversed.

    Args:
        - problem_instance (dict): a dictionary containing key metadata for the defined
            problem instance.
        - parent (dict): the solution selected to be mutated.
        - start_index (int): the chromosome index where the inversion mutation will 
            begin from.

    Returns:
        - offspring (list): a list containing the permutation order of cities for a
            solution. Has length: (problem_instance["dimension"] - 1).
    """
    CHROMOSOME_LENGTH = problem_instance["dimension"] - 1

    offspring = [None for _ in range(CHROMOSOME_LENGTH)]

    # Between 0 and start index, copy the parent unedited
    if start_index != 0:
        for i in range(0, start_index):
            offspring[i] = parent[i]

    # Generate inversion end index (random length between 2 and 10)
    if CHROMOSOME_LENGTH - start_index < 10:
        inversion_length = np.random.randint(2, CHROMOSOME_LENGTH - start_index + 1) 
    else:
        inversion_length = np.random.randint(2, 10) 
    end_index = start_index + inversion_length

    # Between start and end indices, inverse the order of the cities
    for i in range(inversion_length):
        offspring[start_index + i] = parent[end_index - (i + 1)]
    
    # Between end index and the end of chromosome, copy the parent unedited
    for i in range(end_index, CHROMOSOME_LENGTH):
        offspring[i] = parent[i]
    
    return offspring



def perform_swap_sequence_mutation(problem_instance, parent, start_index):
    """
    Used to take a single parent chromosome and perform a swap sequence mutation, returning
    the offspring solution as a result.

    The swap sequence mutation is initiated at start_index. A random length and midpoint are 
    generated, and from that an end_index calculated. The cities between start_index and the
    midpoint are swapped with the cities between the midpoint and end_index.

    Args:
        - problem_instance (dict): a dictionary containing key metadata for the defined
            problem instance.
        - parent (dict): the solution selected to be mutated.
        - start_index (int): the chromosome index where the inversion mutation will 
            begin from.

    Returns:
        - offspring (list): a list containing the permutation order of cities for a
            solution. Has length: (problem_instance["dimension"] - 1).
    """
    CHROMOSOME_LENGTH = problem_instance["dimension"] - 1

    offspring = [None for _ in range(CHROMOSOME_LENGTH)]

    # Between 0 and start index, copy the parent unedited
    if start_index != 0:
        for i in range(0, start_index):
            offspring[i] = parent[i]

    # Generate total change end index (random length between 2 and 10)
    if CHROMOSOME_LENGTH - start_index < 10:
        total_change_length = np.random.randint(2, CHROMOSOME_LENGTH - start_index + 1) 
    else:
        total_change_length = np.random.randint(2, 10) 
    end_index = start_index + total_change_length
    random_midpoint = start_index + np.random.randint(1, total_change_length)

    # Extract the two segments of chromosome from the start index, midpoint and end index
    first_segment = parent[start_index: random_midpoint]
    second_segment = parent[random_midpoint: end_index]

    # Continue offspring by adding second segment
    for i in range(len(second_segment)):
        offspring[start_index + i] = second_segment[i]
    
    # Continue offspring by adding first segment
    for i in range(len(first_segment)):
        offspring[start_index + len(second_segment) + i] = first_segment[i]
    
    # Between end index and the end of chromosome, copy the parent unedited
    for i in range(end_index, CHROMOSOME_LENGTH):
        offspring[i] = parent[i]

    return offspring


In [33]:
def two_opt_local_search(problem_instance, solution):
    """
    Used to optimise a single solution via performing repeat two-opt swaps. 
    
    This process iterates through all possible two-opt swaps, updating the new best 
    solution if the swap improves the solution cost. Once performing two-opt swaps no 
    longer can improve the cost, the best solution is returned. 

    Args:
        - problem_instance (dict): a dictionary containing key metadata for the defined
            problem instance.
        - solution (dict): the solution selected to be optimised via local search.

    Returns:
        - best_solution (dict):
            - chromosome (list): a list containing the permutation order of cities for 
                a solution. Has length: (problem_instance["dimension"] - 1).
            - routes (list[lists]): a list of lists containing the decoded set of routes 
                that is unique to a chromosome.
            - k (int): the number of routes, equal to len(routes).
            - cost (float): the total distance travelled by the routes unique to a 
                solution chromosome.
    """
    DIMENSION = problem_instance["dimension"]

    # Define the initial solution and cost as the best solution and cost
    best_solution = solution.copy()

    improved = True

    while improved:
        improved = False
        for i, j in itertools.combinations(range(1, DIMENSION - 1), 2): 
            # Do not evaluate 2-opt swap on adjacent edges
            if j - i == 1:  
                continue
            
            # Create a new route with reversed segment
            new_chromosome = best_solution["chromosome"][:i] + best_solution["chromosome"][i:j][::-1] + best_solution["chromosome"][j:]
            # Decode and evaluate the new solution
            new_routes, new_k = decode_solution(problem_instance, new_chromosome)
            new_cost = evaluate_solution(problem_instance, new_routes)
            
            # If the cost is lowered, accept the new solution
            if new_cost < best_solution["cost"]:
                best_solution["chromosome"] = new_chromosome
                best_solution["routes"] = new_routes
                best_solution["k"] = new_k
                best_solution["cost"] = new_cost

                improved = True
    
    return best_solution


In [34]:
def genetic_algorithm(problem_instance, pop_size, n_generations, elitism_rate, tournament_bias, 
                      mutation_rate, local_search_volume, local_search_frequency, random_seed):
    """
    Used to perform a single iteration of genetic algorithm.

    Args:
        - problem_instance (dict): a dictionary containing key metadata for the defined
            problem instance. 
        - pop_size (int): the number of solutions to be generated for the population.
        - n_generations (int): the number of generations the GA will iterate for.
        - elitism_rate (float): the proportion of top solutions each generation that are
            designated as elites.
        - tournament_bias (float): the preference towards selecting the fitter solution as 
            the second parent in the biased binary tournament.
        - mutation_rate (float): the probability that a mutation will be initiated at each
            city index.
        - local_search_volume (int): the number of solutions selected at random for optimisation
            via a two-opt swap local search heuristic.
        - local_search_frequency (int): the number of generations between each performance of 
            the two-opt swap local search heuristic.
        - random_seed (int): defines the set seed for the run.

    Returns:
        - best_solution (dict):
            - chromosome (list): a list containing the permutation order of cities for 
                a solution. Has length: (problem_instance["dimension"] - 1).
            - routes (list[lists]): a list of lists containing the decoded set of routes 
                that is unique to a chromosome.
            - k (int): the number of routes, equal to len(routes).
            - cost (float): the total distance travelled by the routes unique to a 
                solution chromosome.
        - progress (list[[generation_no, best_solution]]): a list of the best current solution
            at each generation of the GA runtime.
        - runtime (float): a measure of the time taken for the GA to return the optimised result.
    """
    # Measure the time it takes to complete a run of the GA
    start_time = time.time()

    # Initialise list for storing the progress of the GA across generations
    progress = []

    CHROMOSOME_LENGTH = problem_instance["dimension"] - 1

    # Set seed
    np.random.seed(random_seed)

    # Generate the greedy nearest-neighbour portion of the initial population
    greedy_population = generate_NN_population(problem_instance)

    # Generate the remainder of the initial population randomly
    random_population_size = pop_size - len(greedy_population)
    random_population = generate_random_population(problem_instance, random_population_size)

    initial_population = greedy_population + random_population

    # Sort the initial population from lowest to highest cost
    ranked_population = sorted(initial_population, key = lambda x: x["cost"], reverse = False)

    for i in range(1, n_generations + 1):
        next_gen_population = []

        # ---------
        #  ELITISM
        # ---------

        # Take top percent of unique individuals as elites
        number_of_elites = int(np.ceil(pop_size * elitism_rate))

        # Initialise list of elites
        elites = [ranked_population[0]]
        elites_indexes = [0]

        # Ensures that each of the added elites are unique
        added_elites = 1
        potential_elite_index = 1
        while added_elites < number_of_elites:
            # Check whether next potential elite is identical to an existing selected elite
            elite_match = False
            for elite in elites:
                next_potential_elite = ranked_population[potential_elite_index]
                if next_potential_elite == elite:
                    elite_match = True
                    break
            
            # If there is no match, add the potential elite to the elites list
            if elite_match == False:
                elites.append(ranked_population[potential_elite_index])
                elites_indexes.append(potential_elite_index)
                added_elites += 1
                potential_elite_index += 1
            else: 
                potential_elite_index += 1

        # Carry elite individuals forward unedited
        for elite in elites:
            next_gen_population.append(elite)
        
        # All indexes not in elites are defacto the non elites
        non_elites_indexes = list(set(range(pop_size)) - set(elites_indexes))

        # Collect group of non_elites
        non_elites = []
        for index in non_elites_indexes:
            non_elites.append(ranked_population[index])

        # Fill the remainder of the next generation population with new offspring 
        next_gen_pop_size = number_of_elites
        while next_gen_pop_size < pop_size:

            # ------------------
            #  PARENT SELECTION 
            # ------------------

            parent_1, parent_2 = parent_selection(problem_instance, pop_size, tournament_bias,
                                                    number_of_elites, elites, non_elites)
            
            # -----------
            #  CROSSOVER 
            # -----------
            
            offspring = perform_crossover(problem_instance, parent_1, parent_2)

            # ----------
            #  MUTATION 
            # ----------

            for m in range(len(offspring) - 1): # - 1, as cannot invert only the last allele
                # At each allele, begin a mutation at probability defined by the mutation rate
                if np.random.random() < mutation_rate:
                    # 50% of the time, perform an inversion mutation
                    if np.random.random() < 0.5:
                        offspring = perform_inversion_mutation(problem_instance, offspring, m)
                    # 50% of the time, perform a swap sequence mutation
                    else:
                        offspring = perform_swap_sequence_mutation(problem_instance, offspring, m)
                else: 
                    continue

            # --------------------------------
            # ADD SOLUTION TO NEXT GENERATION
            # --------------------------------

            # Check whether the offspring solution already exists in the next gen population
            solution_match = False
            for next_gen_solution in next_gen_population:
                if offspring == next_gen_solution["chromosome"]:
                    solution_match = True

            # If solution is unique, evaluate it and add it to the next generation
            if solution_match == False:
                # Create solution
                solution = {
                    "chromosome": offspring,
                    "routes": "",
                    "k": "",
                    "cost": ""
                }

                # Decode chromosome into its routes and k value
                routes, k = decode_solution(problem_instance, solution["chromosome"])
                solution["routes"] = routes
                solution["k"] = k

                # Evaluate the cost of the route plan
                cost = evaluate_solution(problem_instance, routes)
                solution["cost"] = cost

                # Add it to the next generation
                next_gen_population.append(solution)
                next_gen_pop_size += 1
            else:
                # If solution matches another, repeat the solution generation process
                continue

        # ----------------------
        # PERIODIC LOCAL SEARCH
        # ----------------------

        # Perform local search only every local_search_frequency generations
        if i % local_search_frequency == 0:
            print("Optimising via Local Search...")

            # Generate local_search_volume number of individuals to improve
            selected_solution_ids = np.random.choice(list(range(0, pop_size)), local_search_volume, replace = False)

            for solution_id in selected_solution_ids:
                solution = next_gen_population[solution_id]

                optimised_solution = two_opt_local_search(problem_instance, solution)

                next_gen_population[solution_id] = optimised_solution

        # --------
        # RANKING
        # --------

        # Sort the population from lowest to highest cost
        ranked_population = sorted(next_gen_population, key = lambda x: x["cost"], reverse = False)
        best_solution = ranked_population[0] # The current best solution

        # Update the GA progress tracker
        generation_progress = [i, best_solution]
        progress.append(generation_progress)

        # Calculate elapsed time
        elapsed_time = time.time() - start_time

        print(f"Generation: {i}, Best cost so far: {ranked_population[0]["cost"]}, Time elapsed: {elapsed_time}")

    progress = np.array(progress)

    end_time = time.time()
    runtime = end_time - start_time

    return best_solution, progress, runtime


## Problem Instances

In [51]:
# Load selected problem instances
X_n101_k25 = load_problem_instance("X-n101-k25.vrp")
X_n120_k6 = load_problem_instance("X-n120-k6.vrp")
X_n172_k51 = load_problem_instance("X-n172-k51.vrp")

## Repeat Results

In [53]:
def GA_repeats_wrapper(problem_instance, pop_size, n_generations, elitism_rate, tournament_bias, 
                       mutation_rate, local_search_volume, local_search_frequency, random_seed, n_repeats):
    """
    Used to perform several iterations of the defined genetic algorithm.

    Args:
        - problem_instance (dict): a dictionary containing key metadata for the defined
            problem instance. 
        - pop_size (int): the number of solutions to be generated for the population.
        - n_generations (int): the number of generations the GA will iterate for.
        - elitism_rate (float): the proportion of top solutions each generation that are
            designated as elites.
        - tournament_bias (float): the preference towards selecting the fitter solution as 
            the second parent in the biased binary tournament.
        - mutation_rate (float): the probability that a mutation will be initiated at each
            city index.
        - local_search_volume (int): the number of solutions selected at random for optimisation
            via a two-opt swap local search heuristic.
        - local_search_frequency (int): the number of generations between each performance of 
            the two-opt swap local search heuristic.
        - random_seed (int): defines the set seed for the run.
        - n_repeats (int): the number of times that optimisation should be performed.

    Returns:
        - repeats (dict):
            - repeat (int): which repeat experiment the result belongs to.
            - best_solution (dict):
                - chromosome (list): a list containing the permutation order of cities for 
                    a solution. Has length: (problem_instance["dimension"] - 1).
                - routes (list[lists]): a list of lists containing the decoded set of routes 
                    that is unique to a chromosome.
                - k (int): the number of routes, equal to len(routes).
                - cost (float): the total distance travelled by the routes unique to a 
                    solution chromosome.
            - progress (list[[generation_no, best_solution]]): a list of the best current solution
                at each generation of the GA runtime.
            - runtime (float): a measure of the time taken for the GA to return the optimised 
                result.
    """
    # Generate n_repeats random seeds
    np.random.seed(random_seed)
    repeat_seeds = np.random.choice(list(range(1, 101)), n_repeats)

    repeats = []

    for i in range(n_repeats):
        print(f"Performing repeat No. {i+1}...")
        # Perform single optimisation run
        best_solution, progress, runtime = genetic_algorithm(problem_instance = problem_instance, 
                                                             pop_size = pop_size,
                                                             n_generations = n_generations, 
                                                             elitism_rate = elitism_rate,
                                                             tournament_bias = tournament_bias,
                                                             mutation_rate = mutation_rate,
                                                             local_search_volume = local_search_volume,
                                                             local_search_frequency = local_search_frequency,
                                                             random_seed = repeat_seeds[i])
        
        # Store run data as a single object
        repeat = {
            "repeat": i + 1,
            "best_solution": best_solution,
            "progress": progress,
            "runtime": runtime
        }

        repeats.append(repeat)
    
    return repeats
